# PII 마스킹 with vLLM (OpenAI 호환 API)

## 목적
- CSV 대화 데이터에서 개인정보(PII) 마스킹
- 외부 API(Judge)에 안전하게 전달하기 위한 전처리

## 마스킹 대상
1. **직접 식별정보**: 이름, 이메일, 전화번호, 주소, 주민등록번호
2. **민감정보(건강)**: 질병/진단명, 검사 수치, 검진 결과, 검진 날짜, 복용 약물

## 1. 환경 설정

In [3]:
import os
import re
import pandas as pd
import requests
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

VLLM_BASE_URL = os.getenv("VLLM_BASE_URL", "http://localhost:8000/v1")
VLLM_API_KEY = os.getenv("VLLM_API_KEY", "EMPTY")
MODEL_NAME = os.getenv("VLLM_MODEL", "openai/gpt-oss-20b")

print(f"vLLM Base URL: {VLLM_BASE_URL}")
print(f"Model: {MODEL_NAME}")

vLLM Base URL: http://192.168.0.86:8000/v1
Model: gaunernst/gemma-3-27b-it-int4-awq


In [4]:
# vLLM 서버 연결 확인 (OpenAI 호환 API)
try:
    response = requests.get(f"{VLLM_BASE_URL}/models")
    if response.status_code == 200:
        models = response.json().get("data", [])
        model_ids = [m["id"] for m in models]
        print("vLLM 서버 연결 성공!")
        print(f"사용 가능한 모델: {model_ids}")
        if not any(MODEL_NAME in m for m in model_ids):
            print(f"\n[경고] {MODEL_NAME} 모델이 없습니다.")
except requests.exceptions.ConnectionError:
    print("vLLM 서버에 연결할 수 없습니다.")
    print("서버 상태를 확인하세요.")

vLLM 서버 연결 성공!
사용 가능한 모델: ['gaunernst/gemma-3-27b-it-int4-awq']


## 2. LLM 설정 (vLLM - OpenAI 호환)

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

llm = ChatOpenAI(
    model=MODEL_NAME,
    base_url=VLLM_BASE_URL,
    api_key=VLLM_API_KEY,
    temperature=0.1,
)

print(f"{MODEL_NAME} 모델 설정 완료!")

gaunernst/gemma-3-27b-it-int4-awq 모델 설정 완료!


## 3. 마스킹 프롬프트 및 함수 정의

In [6]:
MASKING_SYSTEM_PROMPT = """당신은 개인정보 마스킹 전문가입니다.
주어진 텍스트에서 개인정보를 찾아 플레이스홀더로 대체하세요.

## 마스킹 대상 및 플레이스홀더

### 1. 직접 식별정보
- 이름 (사람 이름) -> [NAME]
- 이메일 주소 -> [EMAIL]
- 전화번호 -> [PHONE]
- 주소 -> [ADDRESS]
- 주민등록번호 -> [SSN]

### 2. 민감정보 (건강)
- 질병/진단명 (예: 당뇨, 고혈압, 지방간 등) -> [CONDITION]
- 검사 수치 (예: 115mg/dL, 130/85mmHg 등) -> [HEALTH_VALUE]
- 검진 결과/판정 (예: 정상A, 정상B, 주의 등) -> [RESULT]
- 특정 검진 날짜 (예: 2024년 3월 15일) -> [DATE]
- 복용 약물 -> [MEDICATION]

## 규칙
1. 마스킹 대상만 플레이스홀더로 대체하고, 나머지 텍스트는 그대로 유지하세요.
2. 같은 유형의 정보는 동일한 플레이스홀더를 사용하세요.
3. 나이대(30대), 성별(여성) 같은 일반적 정보는 마스킹하지 마세요.
4. 일반적인 건강 조언, 음식명, 운동명은 마스킹하지 마세요.
5. 마스킹된 텍스트만 출력하세요. 설명이나 주석은 추가하지 마세요.

## 예시

입력: 홍길동님, 2024년 3월 15일 건강검진 결과 공복혈당이 115mg/dL로 고혈압이 경계 수준입니다.
출력: [NAME]님, [DATE] 건강검진 결과 공복혈당이 [HEALTH_VALUE]로 [CONDITION]이 경계 수준입니다.

입력: example123@company.com 계정으로 로그인하셨습니다.
출력: [EMAIL] 계정으로 로그인하셨습니다.
"""

In [7]:
# PII 탐지를 위한 정규식 패턴
PII_PATTERNS = {
    "korean_name": r'[가-힣]{2,4}님',  # 한국인 이름 + 님
    "email": r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}',
    "phone": r'01[0-9]-?\d{3,4}-?\d{4}',
    "date_korean": r'\d{4}년\s*\d{1,2}월\s*\d{1,2}일',
    "date_slash": r'\d{4}[/-]\d{1,2}[/-]\d{1,2}',
    "health_value": r'\d+(\.\d+)?\s*(mg/dL|mmHg|g/dL|%|kg|cm)',
    "ssn": r'\d{6}-?\d{7}',
    "address": r'(서울|부산|대구|인천|광주|대전|울산|세종|경기|강원|충북|충남|전북|전남|경북|경남|제주)[시도]?\s*[가-힣]+[시군구]',
}


def has_pii_pattern(text: str) -> bool:
    """텍스트에 PII 패턴이 있는지 검사"""
    for pattern_name, pattern in PII_PATTERNS.items():
        if re.search(pattern, text):
            return True
    return False


def create_masking_prompt(text: str) -> str:
    """마스킹 요청 프롬프트 생성"""
    return f"""다음 텍스트에서 개인정보를 마스킹하세요.
마스킹할 개인정보가 없으면 텍스트를 그대로 출력하세요.

텍스트:
{text}

마스킹된 텍스트:"""


def validate_masked_result(original: str, masked: str) -> bool:
    """마스킹 결과가 유효한지 검증 (환각 방지)"""
    # 원본이 짧은데 결과가 너무 길면 환각으로 판단
    if len(original) < 50 and len(masked) > len(original) * 3:
        return False
    
    # 결과에 원본에 없던 플레이스홀더가 너무 많으면 환각
    placeholders = ["[NAME]", "[EMAIL]", "[PHONE]", "[ADDRESS]", "[SSN]",
                   "[CONDITION]", "[HEALTH_VALUE]", "[RESULT]", "[DATE]", "[MEDICATION]"]
    placeholder_count = sum(1 for ph in placeholders if ph in masked)
    
    # 원본이 짧은데 플레이스홀더가 5개 이상이면 환각
    if len(original) < 100 and placeholder_count >= 5:
        return False
    
    return True


def mask_text(text: str) -> str:
    """텍스트 마스킹 수행 (PII 패턴 사전 검증 포함)"""
    if not text or pd.isna(text):
        return text
    
    text = str(text).strip()
    
    # 1. PII 패턴이 없으면 원본 그대로 반환
    if not has_pii_pattern(text):
        return text
    
    try:
        messages = [
            SystemMessage(content=MASKING_SYSTEM_PROMPT),
            HumanMessage(content=create_masking_prompt(text))
        ]
        response = llm.invoke(messages)
        masked_result = response.content.strip()
        
        # 2. 환각 검증: 결과가 비정상이면 원본 반환
        if not validate_masked_result(text, masked_result):
            print(f"[경고] 환각 감지, 원본 유지: {text[:50]}...")
            return text
        
        return masked_result
    except Exception as e:
        print(f"Error: {e}")
        return text


# 테스트: PII 패턴 검사
test_cases = [
    "건강리포트",
    "표로 만들어 주면 좋겠어",
    "김영수님, 2024년 5월 20일 검진 결과입니다.",
    "혈압이 130/85mmHg입니다.",
]

print("=== PII 패턴 검사 테스트 ===")
for tc in test_cases:
    has_pii = has_pii_pattern(tc)
    print(f"[{'O' if has_pii else 'X'}] {tc}")

=== PII 패턴 검사 테스트 ===
[X] 건강리포트
[X] 표로 만들어 주면 좋겠어
[O] 김영수님, 2024년 5월 20일 검진 결과입니다.
[O] 혈압이 130/85mmHg입니다.


## 4. 마스킹 테스트

In [8]:
test_text = "김영수님, 2024년 5월 20일 건강검진 결과 총콜레스테롤이 245mg/dL로 고지혈증 주의가 필요합니다."

print("=" * 50)
print("[원본]")
print(test_text)
print("=" * 50)
print("[마스킹 결과]")
print(mask_text(test_text))
print("=" * 50)

[원본]
김영수님, 2024년 5월 20일 건강검진 결과 총콜레스테롤이 245mg/dL로 고지혈증 주의가 필요합니다.
[마스킹 결과]
[NAME]님, [DATE] 건강검진 결과 총콜레스테롤이 [HEALTH_VALUE]로 [CONDITION] 주의가 필요합니다.


## 5. CSV 데이터 로드

In [9]:
df = pd.read_csv("MESSAGE_INFO_REAL.csv")

print(f"총 {len(df)}개 메시지")
print(f"세션 수: {df['session_id'].nunique()}개")
df.head()

총 486개 메시지
세션 수: 40개


,#,email,session_id,role,content,created_at
0,1,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,USER,"안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 질...",2025-10-01 06:04:52.205165 +00:00
1,2,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,ASSISTANT,"조미진님, 안녕하세요. 2023년 11월 17일 건강검진 결과를 바탕으로 건강 상태...",2025-10-01 06:05:30.045710 +00:00
2,3,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,USER,신장(콩팥) 개선 방법을 알려줘,2025-10-01 06:05:37.811683 +00:00
3,4,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,ASSISTANT,"## 신장기능 관리를 위한 3가지 핵심 방법\n\n조미진님, 단백뇨 관리를 위해 다...",2025-10-01 06:05:51.403698 +00:00
4,5,whalwls128@idblife.com,4633956e-067a-4803-91c4-463d15b61877,USER,신장기능 식단 관리 방법을 알려줘,2025-10-01 06:06:11.208222 +00:00


## 6. 전체 CSV 마스킹 실행

In [10]:
def mask_dataframe(df: pd.DataFrame, content_column: str = "content") -> pd.DataFrame:
    """DataFrame의 content 컬럼 전체 마스킹"""
    df_masked = df.copy()
    df_masked["email"] = "[EMAIL]"
    
    masked_contents = []
    for idx, row in tqdm(df_masked.iterrows(), total=len(df_masked), desc="마스킹 진행"):
        masked_content = mask_text(row[content_column])
        masked_contents.append(masked_content)
        
        if (idx + 1) % 10 == 0:
            print(f"[{idx + 1}/{len(df_masked)}] 처리 완료")
    
    df_masked[content_column] = masked_contents
    return df_masked

In [11]:
# 테스트: 처음 20개만 처리
# df_masked = mask_dataframe(df.head(20))

# 전체 처리
df_masked = mask_dataframe(df)

마스킹 진행:   2%|▏         | 10/486 [00:33<22:51,  2.88s/it]

[10/486] 처리 완료


마스킹 진행:   4%|▍         | 20/486 [00:53<19:15,  2.48s/it]

[20/486] 처리 완료


마스킹 진행:   6%|▌         | 30/486 [01:48<35:18,  4.65s/it]

...

마스킹 진행:  97%|█████████▋| 470/486 [17:30<01:07,  4.20s/it]

[470/486] 처리 완료


마스킹 진행:  99%|█████████▉| 480/486 [18:08<00:22,  3.79s/it]

[480/486] 처리 완료


마스킹 진행: 100%|██████████| 486/486 [18:23<00:00,  2.27s/it]


## 7. 결과 확인 및 저장

In [12]:
def compare_masking(original_df: pd.DataFrame, masked_df: pd.DataFrame, n: int = 3):
    """마스킹 전후 비교 출력"""
    for i in range(min(n, len(original_df))):
        print(f"{'='*60}")
        print(f"[{i+1}] 원본:")
        print(original_df.iloc[i]["content"][:500])
        print(f"\n[{i+1}] 마스킹:")
        print(masked_df.iloc[i]["content"][:500])
    print("=" * 60)

compare_masking(df, df_masked, n=3)

[1] 원본:
안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 질병, 현재 증상 등을 고려해서 건강 관리 방안을 제안해주세요.

[1] 마스킹:
안녕하세요! 저의 현재 건강 상태를 종합적으로 분석해주세요. 나이, 성별, 기존 질병, 현재 증상 등을 고려해서 건강 관리 방안을 제안해주세요.
[2] 원본:
조미진님, 안녕하세요. 2023년 11월 17일 건강검진 결과를 바탕으로 건강 상태를 자세히 살펴보겠습니다.

지난 몇 년간의 검진 결과를 종합적으로 살펴보니, 전반적인 건강 상태는 안정적으로 유지되고 있습니다. 2017년, 2019년, 2021년 모두 ‘정상 B’ 또는 ‘정상 A’ 판정을 받으셨고, 최근 검진에서도 ‘정상 B’ 판정을 받으셨습니다. 주의가 필요한 항목의 수는 약간의 변화는 있었지만, 크게 증가하거나 악화된 부분은 보이지 않습니다. 꾸준히 건강 관리에 신경 쓰신 덕분이라고 생각합니다.

현재 가장 집중적으로 관리해야 할 부분은 단백뇨입니다. 단백뇨는 소변으로 단백질이 배출되는 현상인데, 신장이 제대로 기능을 하고 있는지 확인하는 중요한 지표입니다. 신장은 우리 몸의 노폐물을 걸러내는 역할을 하기 때문에, 단백뇨가 지속되면 신장 기능 저하로 이어질 수 있습니다.

30대 여성의 경우, 단백뇨가 심해지면 혈압 상승, 부종, 피로감 등의 증상이 나타날 수 있으며, 심한 경

[2] 마스킹:
[NAME]님, 안녕하세요. [DATE] 건강검진 결과를 바탕으로 건강 상태를 자세히 살펴보겠습니다.

지난 몇 년간의 검진 결과를 종합적으로 살펴보니, 전반적인 건강 상태는 안정적으로 유지되고 있습니다. [DATE], [DATE], [DATE] 모두 ‘[RESULT]’ 또는 ‘[RESULT]’ 판정을 받으셨고, 최근 검진에서도 ‘[RESULT]’ 판정을 받으셨습니다. 주의가 필요한 항목의 수는 약간의 변화는 있었지만, 크게 증가하거나 악화된 부분은 보이지 않습니다. 꾸준히 건강 관리에 신경 쓰신 덕분이라고 생각합니다.

현재 가장 

In [13]:
output_path = "MESSAGE_INFO_MASKED.csv"
df_masked.to_csv(output_path, index=False)

print(f"마스킹된 CSV 저장 완료: {output_path}")
print(f"총 {len(df_masked)}개 메시지 처리")

마스킹된 CSV 저장 완료: MESSAGE_INFO_MASKED.csv
총 486개 메시지 처리


## 8. 마스킹 품질 검증

In [14]:
def validate_masking(df_masked: pd.DataFrame) -> dict:
    """마스킹 품질 검증"""
    placeholders = [
        "[NAME]", "[EMAIL]", "[PHONE]", "[ADDRESS]", "[SSN]",
        "[CONDITION]", "[HEALTH_VALUE]", "[RESULT]", "[DATE]", "[MEDICATION]"
    ]
    
    all_content = " ".join(df_masked["content"].astype(str))
    
    stats = {ph: all_content.count(ph) for ph in placeholders}
    
    # 이메일 유출 검사
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    leaked_emails = re.findall(email_pattern, all_content)
    
    return {"placeholder_counts": stats, "leaked_emails": leaked_emails}


result = validate_masking(df_masked)

print("=== 마스킹 검증 결과 ===")
print(f"총 메시지 수: {len(df_masked)}")
print("\n플레이스홀더 사용 통계:")
for ph, count in result["placeholder_counts"].items():
    if count > 0:
        print(f"  {ph}: {count}회")

if result["leaked_emails"]:
    print(f"\n[경고] 이메일 유출: {result['leaked_emails']}")
else:
    print("\n[OK] 이메일 유출 없음")

=== 마스킹 검증 결과 ===
총 메시지 수: 486

플레이스홀더 사용 통계:
  [NAME]: 178회
  [CONDITION]: 41회
  [HEALTH_VALUE]: 244회
  [RESULT]: 50회
  [DATE]: 196회
  [MEDICATION]: 2회

[OK] 이메일 유출 없음


## 9. 요약

### 처리 완료
- 원본 CSV -> 마스킹된 CSV 변환
- vLLM 기반 OpenAI 호환 API 사용

### 환각 방지 로직 추가
```
1. PII 패턴 사전 검사 (정규식)
   - 이름(~님), 이메일, 전화번호, 날짜, 검사수치 등
   - 패턴 없으면 LLM 호출 없이 원본 반환

2. 결과 검증
   - 원본 대비 결과 길이 비교
   - 플레이스홀더 개수 검증
   - 환각 감지 시 원본 유지
```

### 다음 단계
- 마스킹된 데이터로 재현 데이터 생성
- 외부 API (Judge)에 전달하여 평가